<a href="https://colab.research.google.com/github/lehai0609/KagglePlayground/blob/main/Playground_S5E7_Improving_0_973279.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Overview

In this notebook, I'm doing EDA for the dataset and perform a baseline model based on EDA analysis.
It's a very standardized process, just like the dataset provided in this month's challenge

In [1]:
# prompt: Read train.csv & test.csv from folder: /content/drive/MyDrive/Kaggle/Playground S5E7

import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Playground S5E7/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Playground S5E7/test.csv')

In [2]:
train_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [3]:
test_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0


# III. TARGET ENCODED APPROACH

In [4]:
# Verify GPU availability (after restart)
import cupy as cp
import cudf as cd

# Check GPU info
device = cp.cuda.Device()
print(f"GPU ID: {device.id}")
print(f"GPU Memory: {device.mem_info[1] / 1024**3:.1f} GB total")
print(f"GPU Memory Available: {device.mem_info[0] / 1024**3:.1f} GB free")

# Test cuDF
try:
    test_cudf = cd.DataFrame({'test': [1, 2, 3]})
    print("✅ cuDF working correctly")
    print(f"cuDF version: {cd.__version__}")
except Exception as e:
    print(f"❌ cuDF error: {e}")

# Test cuML
try:
    from cuml.preprocessing import LabelEncoder
    print("✅ cuML working correctly")
except Exception as e:
    print(f"❌ cuML error: {e}")

# Test CuPy
try:
    test_array = cp.array([1, 2, 3])
    print("✅ CuPy working correctly")
    print(f"CuPy version: {cp.__version__}")
except Exception as e:
    print(f"❌ CuPy error: {e}")

GPU ID: 0
GPU Memory: 14.7 GB total
GPU Memory Available: 14.6 GB free
✅ cuDF working correctly
cuDF version: 25.02.01
✅ cuML working correctly
✅ CuPy working correctly
CuPy version: 13.3.0


In [5]:
import cudf as cd
import cupy as cp
import pandas as pd
import numpy as np
from cuml.model_selection import train_test_split
from cuml.preprocessing import LabelEncoder as cuLabelEncoder
from cuml.preprocessing import SimpleImputer as cuSimpleImputer
from cuml.metrics import roc_auc_score as cu_roc_auc_score
from cuml.ensemble import RandomForestClassifier as cuRandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print(f"GPU memory: {cp.cuda.Device().mem_info}")
print("RAPIDS cuDF and cuML loaded successfully")

# Define constants
NUMERICAL_FEATURES = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
CATEGORICAL_FEATURES = ['Stage_fear', 'Drained_after_socializing']
FEATURES = NUMERICAL_FEATURES + CATEGORICAL_FEATURES
STATS = ["mean", "count", "std"]
STATS2 = ["mean", "count"]
FOLDS = 5

print("Constants defined successfully")

GPU memory: (15716188160, 15828320256)
RAPIDS cuDF and cuML loaded successfully
Constants defined successfully


In [6]:
test_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0


In [7]:
# Check for problematic data types that cuDF might not support
print("Checking for unsupported data types...")

def check_dataframe_compatibility(df, name):
    print(f"\n--- {name} ---")
    for col in df.columns:
        dtype = df[col].dtype
        unique_vals = df[col].nunique()
        nulls = df[col].isnull().sum()
        print(f"{col}: {dtype} | Unique: {unique_vals} | Nulls: {nulls}")

        # Check for object columns with mixed types
        if dtype == 'object':
            sample_values = df[col].dropna().head(10).tolist()
            print(f"  Sample values: {sample_values}")

check_dataframe_compatibility(train_df, "train_df")
check_dataframe_compatibility(test_df, "test_df")

Checking for unsupported data types...

--- train_df ---
id: int64 | Unique: 18524 | Nulls: 0
Time_spent_Alone: float64 | Unique: 12 | Nulls: 1190
Stage_fear: object | Unique: 2 | Nulls: 1893
  Sample values: ['No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No']
Social_event_attendance: float64 | Unique: 11 | Nulls: 1180
Going_outside: float64 | Unique: 8 | Nulls: 1466
Drained_after_socializing: object | Unique: 2 | Nulls: 1149
  Sample values: ['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No']
Friends_circle_size: float64 | Unique: 16 | Nulls: 1054
Post_frequency: float64 | Unique: 11 | Nulls: 1264
Personality: object | Unique: 2 | Nulls: 0
  Sample values: ['Extrovert', 'Extrovert', 'Introvert', 'Extrovert', 'Extrovert', 'Extrovert', 'Extrovert', 'Extrovert', 'Introvert', 'Extrovert']

--- test_df ---
id: int64 | Unique: 6175 | Nulls: 0
Time_spent_Alone: float64 | Unique: 12 | Nulls: 425
Stage_fear: object | Unique: 2 | Nulls: 598
  Sample values: ['No', 'Yes', 'No

In [8]:
# Convert pandas DataFrames to cuDF for GPU acceleration
train_gpu = cd.from_pandas(train_df)
test_gpu = cd.from_pandas(test_df)

print("Data converted to GPU format")

# Prepare the data on GPU
train_clean = train_gpu.copy()
test_clean = test_gpu.copy()

# Convert target to binary using cuML LabelEncoder
le_target = cuLabelEncoder()
train_clean['Personality_binary'] = le_target.fit_transform(train_clean['Personality'])

# Check class distribution
print("Class distribution:")
personality_counts = train_clean['Personality'].value_counts()
print(f"Extrovert: {personality_counts.iloc[0] / len(train_clean):.3f}")
print(f"Introvert: {personality_counts.iloc[1] / len(train_clean):.3f}")

print(f"Training data shape: {train_clean.shape}")
print(f"Test data shape: {test_clean.shape}")

Data converted to GPU format
Class distribution:
Extrovert: 0.740
Introvert: 0.260
Training data shape: (18524, 10)
Test data shape: (6175, 8)


In [9]:
# Initialize cross-validation and result storage
# Note: StratifiedKFold from sklearn works with cuDF by converting indices
kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
kf2 = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=43)

# Storage for out-of-fold predictions and feature importance
oof_proba = cp.zeros(len(train_clean))
test_proba = cp.zeros(len(test_clean))
feature_importance = []

print("Cross-validation setup complete")

Cross-validation setup complete


In [10]:
# Helper function to handle cuDF groupby operations efficiently
def gpu_target_encode(df, group_col, target_col, stats_list, smoothing_factor=10):
    """Perform target encoding on GPU with smoothing"""
    grouped = df.groupby(group_col)[target_col].agg(stats_list)

    if 'count' in stats_list and 'mean' in stats_list:
        global_mean = df[target_col].mean()
        # Apply Bayesian smoothing
        smoothed_col = f"{group_col}_smoothed"
        grouped[smoothed_col] = (grouped['mean'] * grouped['count'] + global_mean * smoothing_factor) / (grouped['count'] + smoothing_factor)

    return grouped

print("Helper functions defined")

Helper functions defined


In [11]:
# Main nested cross-validation loop with GPU acceleration
for fold, (train_index, valid_index) in enumerate(kf.split(train_clean.to_pandas(), train_clean['Personality_binary'].to_pandas())):
    print(f"\n=== FOLD {fold + 1} ===")

    # Convert indices to cupy arrays for GPU operations
    train_idx = cp.asarray(train_index)
    valid_idx = cp.asarray(valid_index)

    # Split data for this fold on GPU
    X_train = train_clean.iloc[train_index][FEATURES + ['Personality', 'Personality_binary']].copy()
    X_valid = train_clean.iloc[valid_index][FEATURES].copy()
    X_test = test_clean[FEATURES].copy()

    y_train = X_train['Personality_binary']
    y_valid = train_clean.iloc[valid_index]['Personality_binary']

    print(f"Train size: {len(X_train)}, Valid size: {len(X_valid)}")

    # Handle missing values using cuML
    # Numerical features: median imputation
    num_imputer = cuSimpleImputer(strategy='median')
    X_train_num = X_train[NUMERICAL_FEATURES].copy()
    X_train_num_imputed = cd.DataFrame(num_imputer.fit_transform(X_train_num),
                                       columns=NUMERICAL_FEATURES,
                                       index=X_train.index)
    X_train[NUMERICAL_FEATURES] = X_train_num_imputed

    X_valid_num_imputed = cd.DataFrame(num_imputer.transform(X_valid[NUMERICAL_FEATURES]),
                                       columns=NUMERICAL_FEATURES,
                                       index=X_valid.index)
    X_valid[NUMERICAL_FEATURES] = X_valid_num_imputed

    X_test_num_imputed = cd.DataFrame(num_imputer.transform(X_test[NUMERICAL_FEATURES]),
                                      columns=NUMERICAL_FEATURES,
                                      index=X_test.index)
    X_test[NUMERICAL_FEATURES] = X_test_num_imputed

    # CATEGORICAL FEATURES: Use native cuDF operations (GPU-accelerated)
    # Fit imputation values on training data
    cat_fill_values = {}
    for col in CATEGORICAL_FEATURES:
        # Get most frequent value using cuDF operations
        mode_value = X_train[col].value_counts().index[0]
        cat_fill_values[col] = mode_value

    print(f"Categorical fill values: {cat_fill_values}")

    # Apply imputation to all datasets using cuDF fillna (stays on GPU)
    for dataset_name, dataset in [('train', X_train), ('valid', X_valid), ('test', X_test)]:
        for col in CATEGORICAL_FEATURES:
            dataset[col] = dataset[col].fillna(cat_fill_values[col])
    print(f"✅ {dataset_name} categorical imputation completed")

    # Initialize feature storage for this fold
    train_features = X_train[FEATURES].copy()
    valid_features = X_valid.copy()
    test_features = X_test.copy()

    print("Data preparation complete on GPU")

    # INNER LOOP: Create target-encoded features with leakage prevention
    print("Starting inner loop for target encoding on GPU...")

    # Feature Set 1: Individual numerical features target encoding
    fe1_features = {}

    for inner_fold, (train_index2, valid_index2) in enumerate(kf2.split(X_train.to_pandas(), y_train.to_pandas())):
        X_train2 = X_train.iloc[train_index2].copy()
        X_valid2 = X_train.iloc[valid_index2].copy()

        # Create Feature Set 1: Individual numerical target encoding on GPU
        for col in NUMERICAL_FEATURES:
            try:
                # Create bins for continuous variables using cuDF
                X_train2[f'{col}_binned'] = cd.cut(X_train2[col], bins=10, duplicates='drop')

                # Calculate target statistics using GPU
                grouped_stats = gpu_target_encode(X_train2, f'{col}_binned', 'Personality_binary', STATS)

                # Create mapping dictionary for faster lookup
                bin_mapping = {}
                for stat in ['mean', 'count', 'std']:
                    col_name = f"TE1_{col}_{stat}"
                    bin_mapping[col_name] = grouped_stats[stat].to_pandas().to_dict()

                # Add smoothed version
                col_name_smooth = f"TE1_{col}_smoothed"
                bin_mapping[col_name_smooth] = grouped_stats[f'{col}_binned_smoothed'].to_pandas().to_dict()

                # Create bins for validation data
                X_valid2[f'{col}_binned'] = cd.cut(X_valid2[col], bins=X_train2[f'{col}_binned'].cat.categories)

                # Apply encoding to validation data
                global_mean = X_train2['Personality_binary'].mean()
                for stat in ['mean', 'count', 'std', 'smoothed']:
                    col_name = f'TE1_{col}_{stat}'
                    X_valid2[col_name] = X_valid2[f'{col}_binned'].map(bin_mapping[col_name]).fillna(global_mean)

                    # Store for later use
                    if col_name not in fe1_features:
                        fe1_features[col_name] = cp.zeros(len(X_train))
                    fe1_features[col_name][valid_index2] = X_valid2[col_name].values

            except Exception as e:
                print(f"Error processing {col}: {e}")
                continue

    # Add Feature Set 1 to training data
    for col_name, values in fe1_features.items():
        train_features[col_name] = cd.Series(values, index=train_features.index)

    print(f"Feature Set 1 created on GPU: {len(fe1_features)} features")

    # Feature Set 2: Interaction features target encoding (inner loop on GPU)
    fe2_features = {}
    interactions = [
        ('Social_event_attendance', 'Time_spent_Alone'),
        ('Stage_fear', 'Drained_after_socializing'),
        ('Friends_circle_size', 'Going_outside'),
        ('Stage_fear', 'Social_event_attendance'),
        ('Post_frequency', 'Friends_circle_size')
    ]

    for inner_fold, (train_index2, valid_index2) in enumerate(kf2.split(X_train.to_pandas(), y_train.to_pandas())):
        X_train2 = X_train.iloc[train_index2].copy()
        X_valid2 = X_train.iloc[valid_index2].copy()

        for feat1, feat2 in interactions:
            try:
                # Create interaction feature on GPU
                if feat1 in NUMERICAL_FEATURES:
                    X_train2[f'{feat1}_bin'] = cd.cut(X_train2[feat1], bins=5, duplicates='drop')
                    feat1_use = f'{feat1}_bin'
                else:
                    feat1_use = feat1

                if feat2 in NUMERICAL_FEATURES:
                    X_train2[f'{feat2}_bin'] = cd.cut(X_train2[feat2], bins=5, duplicates='drop')
                    feat2_use = f'{feat2}_bin'
                else:
                    feat2_use = feat2

                # Create interaction using cuDF string operations
                interaction_name = f'{feat1}_{feat2}_interaction'
                X_train2[interaction_name] = X_train2[feat1_use].astype(str) + '_' + X_train2[feat2_use].astype(str)

                # Target encode the interaction on GPU
                grouped_stats = gpu_target_encode(X_train2, interaction_name, 'Personality_binary', STATS2, smoothing_factor=5)

                # Create mapping dictionary
                interaction_mapping = {}
                for stat in ['mean', 'count']:
                    col_name = f"TE2_{interaction_name}_{stat}"
                    interaction_mapping[col_name] = grouped_stats[stat].to_pandas().to_dict()

                # Add smoothed version
                col_name_smooth = f"TE2_{interaction_name}_smoothed"
                interaction_mapping[col_name_smooth] = grouped_stats[f'{interaction_name}_smoothed'].to_pandas().to_dict()

                # Apply to validation data
                if feat1 in NUMERICAL_FEATURES:
                    X_valid2[f'{feat1}_bin'] = cd.cut(X_valid2[feat1], bins=X_train2[f'{feat1}_bin'].cat.categories)
                if feat2 in NUMERICAL_FEATURES:
                    X_valid2[f'{feat2}_bin'] = cd.cut(X_valid2[feat2], bins=X_train2[f'{feat2}_bin'].cat.categories)

                X_valid2[interaction_name] = X_valid2[feat1_use].astype(str) + '_' + X_valid2[feat2_use].astype(str)

                # Apply encoding
                global_mean = X_train2['Personality_binary'].mean()
                for stat in ['mean', 'count', 'smoothed']:
                    col_name = f'TE2_{interaction_name}_{stat}'
                    X_valid2[col_name] = X_valid2[interaction_name].map(interaction_mapping[col_name]).fillna(global_mean)

                    # Store for later use
                    if col_name not in fe2_features:
                        fe2_features[col_name] = cp.zeros(len(X_train))
                    fe2_features[col_name][valid_index2] = X_valid2[col_name].values

            except Exception as e:
                print(f"Error processing interaction {feat1}_{feat2}: {e}")
                continue

    # Add Feature Set 2 to training data
    for col_name, values in fe2_features.items():
        train_features[col_name] = cd.Series(values, index=train_features.index)

    print(f"Feature Set 2 created on GPU: {len(fe2_features)} features")

    # Feature Set 3: Non-target statistical features (safe to calculate directly on GPU)
    fe3_count = 0
    for cat_col in CATEGORICAL_FEATURES:
        for num_col in NUMERICAL_FEATURES:
            try:
                # Calculate statistics on training data using cuDF groupby
                grouped = X_train.groupby(cat_col)[num_col].agg(['mean', 'std'])
                grouped.columns = [f"FE3_{cat_col}_{num_col}_{s}" for s in ['mean', 'std']]

                # Calculate median separately as it's not in standard agg
                median_grouped = X_train.groupby(cat_col)[num_col].median()
                median_col_name = f"FE3_{cat_col}_{num_col}_median"

                # Convert to mapping dictionaries for faster lookup
                mapping_dict = {}
                for stat in ['mean', 'std']:
                    col_name = f"FE3_{cat_col}_{num_col}_{stat}"
                    mapping_dict[col_name] = grouped[col_name].to_pandas().to_dict()

                mapping_dict[median_col_name] = median_grouped.to_pandas().to_dict()

                # Apply to all datasets
                for stat in ['mean', 'std', 'median']:
                    col_name = f"FE3_{cat_col}_{num_col}_{stat}"
                    default_val = X_train[num_col].mean() if stat != 'std' else X_train[num_col].std()

                    train_features[col_name] = X_train[cat_col].map(mapping_dict[col_name]).fillna(default_val).astype('float32')
                    valid_features[col_name] = X_valid[cat_col].map(mapping_dict[col_name]).fillna(default_val).astype('float32')
                    test_features[col_name] = X_test[cat_col].map(mapping_dict[col_name]).fillna(default_val).astype('float32')
                    fe3_count += 1

            except Exception as e:
                print(f"Error processing FE3 {cat_col}_{num_col}: {e}")
                continue

    print(f"Feature Set 3 created on GPU: {fe3_count} features")

    # Apply Feature Sets 1 and 2 to validation and test data using full training data
    # Feature Set 1 for validation/test
    for col in NUMERICAL_FEATURES:
        try:
            X_train[f'{col}_binned'] = cd.cut(X_train[col], bins=10, duplicates='drop')
            grouped_stats = gpu_target_encode(X_train, f'{col}_binned', 'Personality_binary', STATS)

            # Create mapping
            bin_mapping = {}
            for stat in ['mean', 'count', 'std']:
                col_name = f'TE1_{col}_{stat}'
                bin_mapping[col_name] = grouped_stats[stat].to_pandas().to_dict()

            bin_mapping[f'TE1_{col}_smoothed'] = grouped_stats[f'{col}_binned_smoothed'].to_pandas().to_dict()

            global_mean = X_train['Personality_binary'].mean()

            # Apply to validation
            X_valid[f'{col}_binned'] = cd.cut(X_valid[col], bins=X_train[f'{col}_binned'].cat.categories)
            for stat in ['mean', 'count', 'std', 'smoothed']:
                col_name = f'TE1_{col}_{stat}'
                valid_features[col_name] = X_valid[f'{col}_binned'].map(bin_mapping[col_name]).fillna(global_mean)

            # Apply to test
            X_test[f'{col}_binned'] = cd.cut(X_test[col], bins=X_train[f'{col}_binned'].cat.categories)
            for stat in ['mean', 'count', 'std', 'smoothed']:
                col_name = f'TE1_{col}_{stat}'
                test_features[col_name] = X_test[f'{col}_binned'].map(bin_mapping[col_name]).fillna(global_mean)

        except Exception as e:
            print(f"Error applying TE1 {col} to valid/test: {e}")
            continue

    # Feature Set 2 for validation/test
    for feat1, feat2 in interactions:
        try:
            # Prepare features for interaction
            if feat1 in NUMERICAL_FEATURES:
                X_train[f'{feat1}_bin'] = cd.cut(X_train[feat1], bins=5, duplicates='drop')
                feat1_use = f'{feat1}_bin'
            else:
                feat1_use = feat1

            if feat2 in NUMERICAL_FEATURES:
                X_train[f'{feat2}_bin'] = cd.cut(X_train[feat2], bins=5, duplicates='drop')
                feat2_use = f'{feat2}_bin'
            else:
                feat2_use = feat2

            interaction_name = f'{feat1}_{feat2}_interaction'
            X_train[interaction_name] = X_train[feat1_use].astype(str) + '_' + X_train[feat2_use].astype(str)

            grouped_stats = gpu_target_encode(X_train, interaction_name, 'Personality_binary', STATS2, smoothing_factor=5)

            # Create mapping
            interaction_mapping = {}
            for stat in ['mean', 'count']:
                col_name = f'TE2_{interaction_name}_{stat}'
                interaction_mapping[col_name] = grouped_stats[stat].to_pandas().to_dict()

            interaction_mapping[f'TE2_{interaction_name}_smoothed'] = grouped_stats[f'{interaction_name}_smoothed'].to_pandas().to_dict()

            global_mean = X_train['Personality_binary'].mean()

            # Apply to validation
            if feat1 in NUMERICAL_FEATURES:
                X_valid[f'{feat1}_bin'] = cd.cut(X_valid[feat1], bins=X_train[f'{feat1}_bin'].cat.categories)
            if feat2 in NUMERICAL_FEATURES:
                X_valid[f'{feat2}_bin'] = cd.cut(X_valid[feat2], bins=X_train[f'{feat2}_bin'].cat.categories)

            X_valid[interaction_name] = X_valid[feat1_use].astype(str) + '_' + X_valid[feat2_use].astype(str)

            for stat in ['mean', 'count', 'smoothed']:
                col_name = f'TE2_{interaction_name}_{stat}'
                valid_features[col_name] = X_valid[interaction_name].map(interaction_mapping[col_name]).fillna(global_mean)

            # Apply to test
            if feat1 in NUMERICAL_FEATURES:
                X_test[f'{feat1}_bin'] = cd.cut(X_test[feat1], bins=X_train[f'{feat1}_bin'].cat.categories)
            if feat2 in NUMERICAL_FEATURES:
                X_test[f'{feat2}_bin'] = cd.cut(X_test[feat2], bins=X_train[f'{feat2}_bin'].cat.categories)

            X_test[interaction_name] = X_test[feat1_use].astype(str) + '_' + X_test[feat2_use].astype(str)

            for stat in ['mean', 'count', 'smoothed']:
                col_name = f'TE2_{interaction_name}_{stat}'
                test_features[col_name] = X_test[interaction_name].map(interaction_mapping[col_name]).fillna(global_mean)

        except Exception as e:
            print(f"Error applying TE2 {feat1}_{feat2} to valid/test: {e}")
            continue

    # Label encode categorical features using cuML
    le_stage = cuLabelEncoder()
    le_drained = cuLabelEncoder()

    train_features['Stage_fear'] = le_stage.fit_transform(train_features['Stage_fear'])
    train_features['Drained_after_socializing'] = le_drained.fit_transform(train_features['Drained_after_socializing'])

    valid_features['Stage_fear'] = le_stage.transform(valid_features['Stage_fear'])
    valid_features['Drained_after_socializing'] = le_drained.transform(valid_features['Drained_after_socializing'])

    test_features['Stage_fear'] = le_stage.transform(test_features['Stage_fear'])
    test_features['Drained_after_socializing'] = le_drained.transform(test_features['Drained_after_socializing'])

    print(f"Total features created on GPU: {train_features.shape[1]}")

    # Train model - using XGBoost with GPU support
    scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])

    # Debug: Check data types
    print("Checking data types in train_features:")
    problematic_cols = []
    for col in train_features.columns:
        dtype = train_features[col].dtype
        if dtype == 'object':
            problematic_cols.append(col)
            print(f"❌ {col}: {dtype}")
        else:
            print(f"✅ {col}: {dtype}")

    print(f"\nFound {len(problematic_cols)} problematic columns")
    print("First few problematic columns:", problematic_cols[:5])

    # Convert cuDF to DMatrix for XGBoost GPU training
    dtrain = xgb.DMatrix(train_features.to_pandas(), label=y_train.to_pandas())
    dvalid = xgb.DMatrix(valid_features.to_pandas(), label=y_valid.to_pandas())
    dtest = xgb.DMatrix(test_features.to_pandas())

    # XGBoost parameters optimized for GPU
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'tree_method': 'gpu_hist',  # GPU acceleration
        'gpu_id': 0,
        'max_depth': 6,
        'learning_rate': 0.02,
        'scale_pos_weight': scale_pos_weight,
        'random_state': 42,
        'verbosity': 0
    }

    # Train with early stopping
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dvalid, 'validation')],
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Make predictions
    valid_pred = model.predict(dvalid)
    test_pred = model.predict(dtest)

    # Store predictions on GPU
    oof_proba[valid_index] = cp.asarray(valid_pred)
    test_proba += cp.asarray(test_pred) / FOLDS

    # Calculate metrics (convert to CPU for sklearn metrics)
    valid_pred_cpu = cp.asnumpy(cp.asarray(valid_pred))
    y_valid_cpu = y_valid.to_pandas().values

    valid_auc = roc_auc_score(y_valid_cpu, valid_pred_cpu)
    valid_f1 = f1_score(y_valid_cpu, (valid_pred_cpu > 0.5).astype(int))
    valid_acc = accuracy_score(y_valid_cpu, (valid_pred_cpu > 0.5).astype(int))

    print(f"Fold {fold + 1} - AUC: {valid_auc:.4f}, F1: {valid_f1:.4f}, Accuracy: {valid_acc:.4f}")

    # Store feature importance
    importance_df = pd.DataFrame({
        'feature': train_features.columns.to_pandas(),
        'importance': model.get_score(importance_type='weight').values() if hasattr(model, 'get_score') else [0] * len(train_features.columns),
        'fold': fold
    })
    feature_importance.append(importance_df)

    # Clear GPU memory
    del dtrain, dvalid, dtest, train_features, valid_features, test_features
    cp.get_default_memory_pool().free_all_blocks()

# Calculate overall CV performance
oof_proba_cpu = cp.asnumpy(oof_proba)
y_train_cpu = train_clean['Personality_binary'].to_pandas().values

overall_auc = roc_auc_score(y_train_cpu, oof_proba_cpu)
overall_f1 = f1_score(y_train_cpu, (oof_proba_cpu > 0.5).astype(int))
overall_acc = accuracy_score(y_train_cpu, (oof_proba_cpu > 0.5).astype(int))

print(f"\n=== OVERALL CV PERFORMANCE (GPU-Accelerated) ===")
print(f"AUC: {overall_auc:.4f}")
print(f"F1: {overall_f1:.4f}")
print(f"Accuracy: {overall_acc:.4f}")


=== FOLD 1 ===
Train size: 14819, Valid size: 3705
Categorical fill values: {'Stage_fear': 'No', 'Drained_after_socializing': 'No'}
✅ test categorical imputation completed
Data preparation complete on GPU
Starting inner loop for target encoding on GPU...
Error processing Time_spent_Alone: CUDF failure at: /__w/cudf/cudf/cpp/src/labeling/label_bins.cu:219: The left and right edge columns cannot contain nulls.
Error processing Social_event_attendance: CUDF failure at: /__w/cudf/cudf/cpp/src/labeling/label_bins.cu:219: The left and right edge columns cannot contain nulls.
Error processing Going_outside: CUDF failure at: /__w/cudf/cudf/cpp/src/labeling/label_bins.cu:219: The left and right edge columns cannot contain nulls.
Error processing Friends_circle_size: CUDF failure at: /__w/cudf/cudf/cpp/src/labeling/label_bins.cu:219: The left and right edge columns cannot contain nulls.
Error processing Post_frequency: CUDF failure at: /__w/cudf/cudf/cpp/src/labeling/label_bins.cu:219: The left

ValueError: feature_names mismatch: ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency', 'Stage_fear', 'Drained_after_socializing', 'TE2_Stage_fear_Drained_after_socializing_interaction_mean', 'TE2_Stage_fear_Drained_after_socializing_interaction_count', 'TE2_Stage_fear_Drained_after_socializing_interaction_smoothed', 'FE3_Stage_fear_Time_spent_Alone_mean', 'FE3_Stage_fear_Time_spent_Alone_std', 'FE3_Stage_fear_Time_spent_Alone_median', 'FE3_Stage_fear_Social_event_attendance_mean', 'FE3_Stage_fear_Social_event_attendance_std', 'FE3_Stage_fear_Social_event_attendance_median', 'FE3_Stage_fear_Going_outside_mean', 'FE3_Stage_fear_Going_outside_std', 'FE3_Stage_fear_Going_outside_median', 'FE3_Stage_fear_Friends_circle_size_mean', 'FE3_Stage_fear_Friends_circle_size_std', 'FE3_Stage_fear_Friends_circle_size_median', 'FE3_Stage_fear_Post_frequency_mean', 'FE3_Stage_fear_Post_frequency_std', 'FE3_Stage_fear_Post_frequency_median', 'FE3_Dr

In [ ]:
# Analyze feature importance (convert to CPU for analysis)
if feature_importance:
    feature_importance_df = pd.concat(feature_importance, ignore_index=True)
    feature_avg_importance = feature_importance_df.groupby('feature')['importance'].mean().sort_values(ascending=False)

    print("Top 20 most important features:")
    print(feature_avg_importance.head(20))

    # Show feature type distribution
    fe1_features = [f for f in feature_avg_importance.index if f.startswith('TE1_')]
    fe2_features = [f for f in feature_avg_importance.index if f.startswith('TE2_')]
    fe3_features = [f for f in feature_avg_importance.index if f.startswith('FE3_')]
    original_features = [f for f in feature_avg_importance.index if f in FEATURES]

    print(f"\nFeature type distribution:")
    print(f"Original features: {len(original_features)}")
    print(f"Target-encoded individual (FE1): {len(fe1_features)}")
    print(f"Target-encoded interactions (FE2): {len(fe2_features)}")
    print(f"Statistical features (FE3): {len(fe3_features)}")
    print(f"Total features: {len(feature_avg_importance)}")
else:
    print("No feature importance data available")

In [ ]:
# Create submission (convert GPU arrays to CPU)
test_proba_cpu = cp.asnumpy(test_proba)
test_predictions_binary = (test_proba_cpu > 0.5).astype(int)

# Convert back to original labels
test_predictions = le_target.inverse_transform(cd.Series(test_predictions_binary)).to_pandas().values

submission_df = pd.DataFrame({
    'id': test_clean['id'].to_pandas(),
    'Personality': test_predictions
})

print("Submission predictions distribution:")
print(submission_df['Personality'].value_counts(normalize=True))

# Save submission
submission_df.to_csv('gpu_target_encoded_submission.csv', index=False)
print("\nSubmission file 'gpu_target_encoded_submission.csv' created successfully.")

# Display sample of submission
display(submission_df.head(10))

# Clear GPU memory
cp.get_default_memory_pool().free_all_blocks()
print("GPU memory cleared")